<h2 style="font-family: monospace; color: purple;"> Poker Player Patterns</h2>

The data needed is [here](https://www.kaggle.com/smeilz/poker-holdem-games#Export%20Holdem%20Manager%202.0%2012292016131233.txt), specific file used is 
<b><span style="color: black; background-color: red;">Export Holdem Manager 2.0 12302016144830.txt</span></b>.

<h3 style="color: green; font-family: monospace;">Environment settings</h3>

In [24]:
from pathlib import Path
import pandas as pd
from lib import ActionId, TurnId

nrounds, nrowdisplay = 10000, 15
pd.options.display.max_rows = nrowdisplay

### Loading csv

In [25]:
load_path = Path.cwd() / 'tidy_data'

action_ids = pd.read_csv(load_path / 'action_ids.csv')
user_ids = pd.read_csv(load_path / 'user_ids.csv')
actions = pd.read_csv(load_path / 'actions.csv')
cardshow = pd.read_csv(load_path / 'cardshow.csv')
buyins = pd.read_csv(load_path / 'buyins.csv')
blinds = pd.read_csv(load_path / 'blinds.csv')

actions = actions[actions.round_id <= nrounds]
cardshow = cardshow[cardshow.round_id <= nrounds]
buyins = buyins[buyins.round_id <= nrounds]
blinds = blinds[blinds.round_id <= nrounds]

action_dict = dict(zip(action_ids.action_name, action_ids.action_id))
user_dict = list(user_ids.user.sort_values())
print(action_dict)

{'folds': 0, 'calls': 1, 'raises': 2, 'checks': 3, 'bets': 4, 'allin': 5}


### Basics
We'll gather some basic information about the players.
For each player we want:
- number of rounds played
- number of each action taken
- relative winnings

In [26]:
n_rounds = buyins.groupby('user_id').apply(
    lambda df: pd.DataFrame(
        pd.DataFrame(
            data = {'rounds_played': [len(df.round_id)]}
        )
    )
).reset_index().drop(columns = ['level_1'])

In [27]:
n_actions = actions.groupby('user_id').apply(
    lambda df: pd.DataFrame(
        data = {
            key: [len(df.action_id[df.action_id == i])]
            for key, i in action_dict.items()
        }
    )
).reset_index().drop(columns = ['level_1'])

In [28]:
winnings = cardshow.groupby('user_id').apply(
    lambda df: pd.DataFrame(
        data = {
            'total': [df.amount.sum()],
            'mean': [df.amount.mean()]
        }
    )
).reset_index().drop(columns = ['level_1'])

In [33]:
mx = winnings.total.max()
player = winnings[winnings.total == mx]
df.merge(user_ids, how='outer', on='user_id').dropna()


,user_id,total,mean,user
0,5,565.01,0.057072,IlxxxlI
